In [ ]:
# ЕДИНЫЙ КОД: АМ + ЧМ + ШУМ + БПФ/ОБПФ + АВТОВЫБОР ВАРИАНТА
import numpy as np
import matplotlib.pyplot as plt

# ================= ПАРАМЕТРЫ =================
N = 3   # номер в списке
k = 1   # коэффициент
Fs = 50000
T = 0.1

# ====== АВТОМАТИЧЕСКИЙ ВЫБОР ЧЕТНОСТИ ======
if N % 2 == 0:  # четный вариант
    A = 15 * N
    B = 1 * N
    f1 = 3 * k * N
    f2 = 100 * N
else:          # нечетный вариант
    A = 10 * N
    B = 2 * N
    f1 = 2 * k * N
    f2 = 150 * N

# ================= ВРЕМЯ =================
t = np.linspace(0, T, int(Fs*T), endpoint=False)

# ================= СИГНАЛЫ =================
carrier = A * np.sin(2*np.pi*f1*t)
info = B * np.sin(2*np.pi*f2*t)

# ---- АМ ----
am_signal = (A + info) * np.sin(2*np.pi*f1*t)

# ---- ЧМ ----
kf = 50
fm_signal = A * np.sin(2*np.pi*f1*t + kf * np.sin(2*np.pi*f2*t))

# ================= ШУМ =================
noise_am = np.random.normal(0, A*0.5, size=len(t))
noise_fm = np.random.normal(0, A*0.5, size=len(t))

am_noisy = am_signal + noise_am
fm_noisy = fm_signal + noise_fm

# ================= БПФ =================
def fft_process(signal):
    fft = np.fft.fft(signal)
    freq = np.fft.fftfreq(len(signal), 1/Fs)
    return freq, fft

freq, fft_am = fft_process(am_noisy)
_, fft_fm = fft_process(fm_noisy)

# ================= ФИЛЬТРАЦИЯ =================
def spectral_filter(fft_signal, level=0.2):
    threshold = np.max(np.abs(fft_signal)) * level
    return np.where(np.abs(fft_signal) > threshold, fft_signal, 0)

fft_am_filt = spectral_filter(fft_am)
fft_fm_filt = spectral_filter(fft_fm)

# ================= ОБРАТНОЕ БПФ =================
am_rec = np.fft.ifft(fft_am_filt).real
fm_rec = np.fft.ifft(fft_fm_filt).real

# ================= ГРАФИКИ =================
plt.figure(figsize=(16,12))

plt.subplot(4,2,1)
plt.plot(t, am_signal)
plt.title("АМ сигнал без шума")

plt.subplot(4,2,2)
plt.plot(t, am_noisy)
plt.title("АМ сигнал с шумом")

plt.subplot(4,2,3)
plt.plot(freq, np.abs(fft_am))
plt.xlim(0, 500)
plt.title("Спектр АМ с шумом")

plt.subplot(4,2,4)
plt.plot(t, am_rec)
plt.title("АМ после фильтрации и ОБПФ")

plt.subplot(4,2,5)
plt.plot(t, fm_signal)
plt.title("ЧМ сигнал без шума")

plt.subplot(4,2,6)
plt.plot(t, fm_noisy)
plt.title("ЧМ сигнал с шумом")

plt.subplot(4,2,7)
plt.plot(freq, np.abs(fft_fm))
plt.xlim(0, 500)
plt.title("Спектр ЧМ с шумом")

plt.subplot(4,2,8)
plt.plot(t, fm_rec)
plt.title("ЧМ после фильтрации и ОБПФ")

plt.tight_layout()
plt.show()
